In [1]:
using Pkg
Pkg.add("IteratorInterfaceExtensions")
using IteratorInterfaceExtensions

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [2]:
import Base: iterate, getindex, size, length

In [3]:
# ZipIterable Type construction
#  - Both inputs need to be non-empty and iterable (see IteratorInterfaceExtensions)
#  - Both inputs do not need to be AbstractArrays and can have different types
struct ZipIterable{T1,T2}
    first     :: T1
    second    :: T2
    # inner constructor
    ZipIterable{T1,T2}(first,second) where {T1,T2} = (IteratorInterfaceExtensions.isiterable(first) && IteratorInterfaceExtensions.isiterable(second)) ? new{T1,T2}(first, second) : error("ZipIterable: Iterable members only") 
end
# friendlier constructor
ZipIterable(first::T1,second::T2) where {T1,T2} = ZipIterable{T1,T2}(first, second)

ZipIterable

In [4]:
function iterate(z::ZipIterable{T}) where {T}
    if(isempty(z.first))
        return nothing
    else
    item,contInd = iterate(z.first)
    (item,(contInd,1))
    end
    
end

function iterate(z::ZipIterable{T},state) where {T}
    if  (state[2] == 1)
        qTup = iterate(z.first,(state[1]))
        if qTup == nothing
            if(isempty(z.second))
                return nothing
            else
            item,contInd = iterate(z.second)
            return (item,(contInd,2))
            end
        else
            return (qTup[1],(qTup[2],1))
        end
    else
        qTup = iterate(z.second,(state[1]))
        if qTup == nothing
            return nothing
        else
            return (qTup[1],(qTup[2],2))
        end
    end
end

iterate (generic function with 207 methods)

# Demo:#

In [8]:
# Can be constructed with any iterable types (does not even have to be AbstractArray, only iterable)
z1 = ZipIterable([5,6,7],[10,11,12,13])
z2 = ZipIterable(5:7,10:16)
z3 = ZipIterable(rand(2,2,2),10:16)
z4 = ZipIterable(z1,z3)

ZipIterable{ZipIterable{Array{Int64,1},Array{Int64,1}},ZipIterable{Array{Float64,3},UnitRange{Int64}}}(ZipIterable{Array{Int64,1},Array{Int64,1}}([5, 6, 7], [10, 11, 12, 13]), ZipIterable{Array{Float64,3},UnitRange{Int64}}([0.173421 0.783099; 0.245782 0.920528]

[0.0389713 0.445583; 0.598243 0.783104], 10:16))

In [9]:
for i in z4
    println(i)
end

5
6
7
10
11
12
13
0.17342115729510343
0.2457818863382799
0.7830985015730503
0.9205279550759919
0.03897132860263275
0.5982431310626704
0.4455825987579731
0.783104292603356
10
11
12
13
14
15
16


In [7]:
@code_typed iterate(z3)

CodeInfo(
1 ── %1  = (Base.getfield)(z, :first)::Array{Float64,3}
│    %2  = (Base.arraylen)(%1)::Int64
│    %3  = (%2 === 0)::Bool
└───       goto #3 if not %3
2 ──       return Main.nothing
3 ── %6  = (Base.getfield)(z, :first)::Array{Float64,3}
│    %7  = (Base.arraylen)(%6)::Int64
│    %8  = (Base.sle_int)(0, %7)::Bool
│    %9  = (Base.bitcast)(UInt64, %7)::UInt64
│    %10 = (Base.ult_int)(0x0000000000000000, %9)::Bool
│    %11 = (Base.and_int)(%8, %10)::Bool
└───       goto #5 if not %11
4 ── %13 = (Base.arrayref)(false, %6, 1)::Float64
│    %14 = (Core.tuple)(%13, 2)::Tuple{Float64,Int64}
└───       goto #6
5 ── %16 = Base.nothing::Nothing
└───       goto #6
6 ┄─ %18 = φ (#4 => %13)::Float64
│    %19 = φ (#4 => 2)::Int64
│    %20 = φ (#4 => %14, #5 => %16)::Union{Nothing, Tuple{Float64,Int64}}
└───       goto #7
7 ── %22 = Base.indexed_iterate::Const(indexed_iterate, false)
│    %23 = (isa)(%20, Tuple{Float64,Int64})::Bool
└───       goto #9 if not %23
8 ── %25 = (Core.tuple)(%18

# random try stuff section:#

In [31]:
#ac = 5:10
ac = [5 6 7 8 9 10]
next = iterate(ac)
while next !== nothing
    println(next)
    (i, state) = next
    # body
    next = iterate(ac, state)
    
end

(5, 2)
(6, 3)
(7, 4)
(8, 5)
(9, 6)
(10, 7)


In [5]:
zz = 10:13
a = ["aa" "bb" "cc"]
item,ind = iterate(a)
q1 = (item,(ind,1))
println(q1)
item, ind = iterate(a,q[2][1])
q2 = (item,(ind,1))
println(q2)

("aa", (2, 1))
("bb", (3, 1))


CodeInfo(
1 ─ %1 = (Base.add_int)(x, y)::Int64
└──      return %1
) => Int64

In [15]:
aa = Dict("item" => 1 , "cont" => (3,2))
typeof(aa)

Dict{String,Any}

In [16]:
aa["cont"]

(3, 2)

In [ ]:
index, container =  getindexTuple(z,state)
        if (container == 1)
            iterate(z.first,state)
            #println((index, container))
        else
            iterate(z.second,state)
            #println((index, container))
        end
    end

In [23]:
function getindexTuple(z::ZipIterable, i::Int)
    if (i <= 0) || (i > length(z))
        throw(BoundsError())
    elseif(i > length(z.first))
        # second container
        (i - length(z.first), 2)
    else
        # first container 
        (i,1)    
    end
end

length(z::ZipIterable) = length(z.second) + length(z.first);
size(z::ZipIterable) = (length(z),) # what if I pass a matrix and a vector?
# The problem, the state of the first and the whole data structure match but the state of the second and the whole datasructure dont match 
function iterate(z::ZipIterable{T}) where {T}
    q = iterate(z.first)
end
function iterate(z::ZipIterable{T},state) where {T}
    if (state > length(z)) 
        nothing
    else   
        q = iterate(z.first,state)
    end
    if q == nothing
        q = iterate(z.second)
    end
end

iterate (generic function with 208 methods)